In [1]:
import pandas as pd
import numpy as np
import openpyxl

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Load Data

In [2]:
global_cs_2023 = pd.read_excel("../Data/Global Case Sales 2023.xlsx")
master_facility = pd.read_excel("../Data/Master Facility List.xlsx")
assurance_volume = pd.read_csv("../Data/QSET_ASSURANCE_PROD_VOL_EXPORT.csv")

water_wwd = pd.read_excel("../Data/Water & WWD Comments.xlsx")
water_wwd = water_wwd[water_wwd['Indicator'] == 'Total Wastewater Discharged (kL) [kL]']

In [3]:
column_names = [
    "Indicator_Name", "Code", "Entity_Name", 
    "Facility_ID", "Reporting_Period", "Answer"
]

numeric_input_ind = pd.read_csv("../Data/Monthly Numeric Indicator.csv", encoding='utf-16', sep='\t', header=0, names=column_names)
numeric_input_ind.head()


,Indicator_Name,Code,Entity_Name,Facility_ID,Reporting_Period,Answer
0,Scope 2 - TCCS - Steam - CO2eq (IPCC 2013) [t ...,RPT.GHG.10.70.50,CELAYA GUANAJUATO - 6606,6606,23-Oct,0.0
1,Scope 2 - TCCS - Steam - CO2eq (IPCC 2013) [t ...,RPT.GHG.10.70.50,CELAYA GUANAJUATO - 6606,6606,24-Mar,0.0
2,Scope 2 - TCCS - Steam - CO2eq (IPCC 2013) [t ...,RPT.GHG.10.70.50,CELAYA GUANAJUATO - 6606,6606,23-Nov,0.0
3,Scope 2 - TCCS - Steam - CO2eq (IPCC 2013) [t ...,RPT.GHG.10.70.50,CELAYA GUANAJUATO - 6606,6606,23-Sep,0.0
4,Scope 2 - TCCS - Steam - CO2eq (IPCC 2013) [t ...,RPT.GHG.10.70.50,CELAYA GUANAJUATO - 6606,6606,23-May,0.0


In [4]:
column_names = [
    "Reporting_Period", "Entity_Name", "Code", "Indicator_Name", "Answer", "Unit", 
    "Help_Text", "Comments", "Frequency", "FolderPath", "Ord"
]
text_input_ind = pd.read_csv("../Data/Monthly Text Input Indicator.csv", encoding='ISO-8859-1',header=0, names=column_names)

### Task 1: Text Input
- Check to see if answers for the same indicator switches one month to next, if yes outlier
- If answer is missing, outlier

- Columns to keep: Facility_ID, Facility_Name, Bottler, OU, Reporting_Period, Detection_Results

In [5]:
master_facility.head(5)

,FACILITY_ID,FACILITY_NAME,FACILITY_NAME_OT,FAC_BU_CD,CTRY_CD2,FAC_TYPE_CD,FAC_TYPE_NM,STREET_ADDRESS,CITY,STATE_PROVINCE,ZIPCODE,LATITUDE,LONGITUDE,LAST_UPDATED,ACTV_DT,INAC_DT,OWNERSHIP_TYPE,COMPANY_OWNED,LAST_UPDATED_BY,OWNR_ID,BTLR_ISSCOM_CD,OWNR_NM,BTLR_LONG_NM,BTLR_NAME_ISSCOM,BTLR_TYPE_COPACK,BTLR_TYPE_CD_ISSCOM,BTLR_TYPE_NAME_ISSCOM,REC_UPDATE_DATE,CTRY_CD3,CTRY_NM,CTRY_OFFL_NM,GRP_NM,GRP_ORG_ID,DIV_ORG_NM,DIV_ORG_ID,DIV_ORG_SHORT_NM,DIV_ORG_BU3,RGN_NM,RGN_ORG_ID,GLN_NUMBER,DEDICATED_KO,CPACK_CNTRCT_ISSCOM_CD,CPACK_CNTRCT_ISSCOM_NM,CPS_GROUP_CD,CPS_GROUP_NM,FAC_AREA_SQMETERS,BTLR_COMMENT,FAC_COMMENT,BTLR_CTRY_CD2_HQ,BTLR_URL_HQ,ACTIVE_INACTIVE,RGNNAME_RGN,BUNAME_LONG_BU,BUNAME_BU,BUNAME_BU3_BU,GRPNAME_GRP,ADDRESS_CITY_STATE,CPS_BU_NM,CPS_RGN_NM,SGP_COLOR_RATNG,SGP_COMPLIANT,KOSMOS_Entity_Name,Location_Category,WATER_STRESS_TIER,WATER_STRESS_CATEGORY,Minor_Basin,Water_Supply_Watershed,2025_LTI_FLAG,2026_LTI_FLAG,SUST_GRP,QSE_1_Scorecard_Exclusions,VENTURE_PARTNER,Equity_Share_(%),MCDONALDS_SUPPLY,REFRESH_DATE,VIRT_AGGR,ALL_FAC_TYPE_CD,ALL_FAC_TYPE_NM,BTLR_LOCATION,SAP_PLANT_CD,VENTURE_PTNR,COPACK_DEDICATED,FAC_OWNED,BTLR_CONSORT,REPORT_UNIT_CD,REPORT_UNIT_NM,2024_LTI_FLAG,BTLR_CONSORT_ID,KOSMOS_SYNC,ASSET_LOC_ID,TCCC_OWNED,NetworkFolder
0,1,ABIDJAN,NaN,NaN,CI,1001,PRODUCTION MANUFACTURING OPERATION,35 RUE DES BRASSEURS,ABIDJAN,NaN,NaN,5.2961,-4.001314,2024-08-09 13:55:33,2017-08-01,2022-06-30,NaN,NaN,Global Technical Audit Management,20079,9000266.0,CASTEL,NaN,BGI,NO,2.0,INDEPENDENT BOTTLER,2025-03-27 22:03:45,CIV,COTE D'IVOIRE,REPUBLIC OF IVORY COAST,"EUROPE, MIDDLE EAST & AFRICA",21000100,AFRICA,21000210,AFRICA,NaN,WEST ISLANDS & MID AFRICA,21000710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INACTIVE,WEST ISLANDS & MID AFRICA RGN,AFRICA BU,AFRICA BU,BU,EMEA GRP,ABIDJAN,CPS AFRICA BU,CPS WEST ISLANDS & MID AFRICA RGN,NaN,NaN,ABIDJAN - 1,NaN,Tier 3,Low,Agneby,NaN,No,No,NaN,No,No,NaN,No,2025-04-20,NaN,1001,PRODUCTION MANUFACTURING OPERATION,NSR ID,SAP PLANT CODE,NO,NOT A CO-PACKER,OWNED/LEASED/THIRD PARTY,NaN,NaN,NaN,No,NaN,YES,NaN,NO,https://groups.coca-cola.com/:f:/r/sites/GMPPr...
1,10006,CALGARY,NaN,NaN,CA,1001,PRODUCTION MANUFACTURING OPERATION,4810 76 AVENUE SE,CALGARY,ALBERTA,NaN,50.985322,-113.965567,2024-08-09 13:39:49,2017-08-01,2018-01-24,NaN,NaN,Global Technical Audit Management,20364,NaN,REFRESCO,NaN,NaN,YES,NaN,NaN,2025-03-27 22:03:46,CAN,CANADA,CANADA,NORTH AMERICA,11300000,NORTH AMERICA,11300500,NaN,NaN,CANADA,11300302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INACTIVE,CANADA RGN,NORTH AMERICA BU,NA BU,BU,NA GRP,CALGARY ALBERTA,CPS NA BU,CPS CANADA RGN,NaN,NaN,Calgary - 10006,NaN,NaN,NaN,NaN,NaN,No,No,NaN,No,No,NaN,NaN,2025-04-20,NaN,1001,PRODUCTION MANUFACTURING OPERATION,NSR ID,SAP PLANT CODE,NO,NON-DEDICATED,OWNED/LEASED/THIRD PARTY,NaN,NaN,NaN,No,NaN,YES,NaN,NO,https://groups.coca-cola.com/:f:/r/sites/GMPPr...
2,10007,CCNA RTM NON PRODUCTION,NaN,NaN,US,1004,VIRTUAL PHYSICAL LOCATION REPORTING,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-10 13:51:47,2017-08-01,2020-09-01,NaN,NaN,Eddie Caamano,20097,9004561.0,CCNA,NaN,CCNA BOTTLING,NO,4.0,OTHER CONS OPS - NON CBO,2025-01-28 17:43:22,USA,UNITED STATES,UNITED STATES,NORTH AMERICA,11300000,NORTH AMERICA,11300500,NaN,NaN,USA,11300205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INACTIVE,USA RGN,NORTH AMERICA BU,NA BU,BU,NA GRP,NaN,CPS NA BU,CPS USA RGN,NaN,NaN,CCNA RTM NON PRODUCTION - 10007,NaN,NaN,NaN,NaN,NaN,No,No,NaN,No,No,NaN,NaN,2025-04-20,NaN,1004,VIRTUAL PHYSICAL LOCATION REPORTING,NSR ID,SAP PLANT CODE,NO,NOT A CO-PACKER,OWNED/LEASED/THIRD PARTY,CCBSS,NaN,NaN,No,218.0,YES,NaN,NO,https://groups.coca-cola.com/:f:/r/sites/GMPPr...
3,10008,LOGISTIC REGION WEST,NaN,NaN,DE,1004,VIRTUAL PHYSICAL LOCATION REPORTING,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-10 13:51:48,2017-08-01,2019-10-01,NaN,NaN,Eddie Caamano,20125,9007505.0,COCA-COLA EUROPACIFIC PARTNERS,NaN,COCA-COLA EUROPEAN PARTNERS,NO,3.0,EQUITY BOTTLER,2024-10-15 22:02:47,DEU,GERMANY,FEDERAL REPUBLIC OF GERMANY,"EUROPE

In [6]:
master_facility.columns

Index(['FACILITY_ID', 'FACILITY_NAME', 'FACILITY_NAME_OT', 'FAC_BU_CD',
       'CTRY_CD2', 'FAC_TYPE_CD', 'FAC_TYPE_NM', 'STREET_ADDRESS', 'CITY',
       'STATE_PROVINCE', 'ZIPCODE', 'LATITUDE', 'LONGITUDE', 'LAST_UPDATED',
       'ACTV_DT', 'INAC_DT', 'OWNERSHIP_TYPE', 'COMPANY_OWNED',
       'LAST_UPDATED_BY', 'OWNR_ID', 'BTLR_ISSCOM_CD', 'OWNR_NM',
       'BTLR_LONG_NM', 'BTLR_NAME_ISSCOM', 'BTLR_TYPE_COPACK',
       'BTLR_TYPE_CD_ISSCOM', 'BTLR_TYPE_NAME_ISSCOM', 'REC_UPDATE_DATE',
       'CTRY_CD3', 'CTRY_NM', 'CTRY_OFFL_NM', 'GRP_NM', 'GRP_ORG_ID',
       'DIV_ORG_NM', 'DIV_ORG_ID', 'DIV_ORG_SHORT_NM', 'DIV_ORG_BU3', 'RGN_NM',
       'RGN_ORG_ID', 'GLN_NUMBER', 'DEDICATED_KO', 'CPACK_CNTRCT_ISSCOM_CD',
       'CPACK_CNTRCT_ISSCOM_NM', 'CPS_GROUP_CD', 'CPS_GROUP_NM',
       'FAC_AREA_SQMETERS', 'BTLR_COMMENT', 'FAC_COMMENT', 'BTLR_CTRY_CD2_HQ',
       'BTLR_URL_HQ', 'ACTIVE_INACTIVE', 'RGNNAME_RGN', 'BUNAME_LONG_BU',
       'BUNAME_BU', 'BUNAME_BU3_BU', 'GRPNAME_GRP', 'ADDRESS_CIT

In [7]:
master_facility.dtypes

FACILITY_ID                            int64
FACILITY_NAME                         object
FACILITY_NAME_OT                      object
FAC_BU_CD                             object
CTRY_CD2                              object
FAC_TYPE_CD                            int64
FAC_TYPE_NM                           object
STREET_ADDRESS                        object
CITY                                  object
STATE_PROVINCE                        object
ZIPCODE                               object
LATITUDE                              object
LONGITUDE                             object
LAST_UPDATED                  datetime64[ns]
ACTV_DT                       datetime64[ns]
INAC_DT                       datetime64[ns]
OWNERSHIP_TYPE                        object
COMPANY_OWNED                        float64
LAST_UPDATED_BY                       object
OWNR_ID                                int64
BTLR_ISSCOM_CD                       float64
OWNR_NM                               object
BTLR_LONG_

In [8]:
text_input_ind['Facility_ID'] = text_input_ind['FolderPath'].apply(lambda x: x.split('>')[-1].strip())
# Convert Facility_ID to int64
text_input_ind['Facility_ID'] = pd.to_numeric(text_input_ind['Facility_ID'], errors='coerce').astype('int64')
text_input_ind.head()

,Reporting_Period,Entity_Name,Code,Indicator_Name,Answer,Unit,Help_Text,Comments,Frequency,FolderPath,Ord,Facility_ID
0,1/1/2024,REYES COCA-COLA BOTTLING RTM NON PRODUCTION - ...,FLT.10.10.10,Fleet Vehicle Classification,Owned and Leased Distribution,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 11300000 > 11300500 > 11300205 > ...,01/01/24-10011-0030-0010-0010-0010,10011
1,1/1/2024,HCCB SANAND - 10013,FLT.10.10.10,Fleet Vehicle Classification,Third Party Distribution Vehicles|Owned and Le...,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,01/01/24-10013-0030-0010-0010-0010,10013
2,1/1/2024,TCCC RTM NON PRODUCTION - 10031,FLT.10.10.10,Fleet Vehicle Classification,Not Applicable,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 11300000 > 11300500 > 11300205 > ...,01/01/24-10031-0030-0010-0010-0010,10031
3,1/1/2024,DUQUE DE CAXIAS - 10053,FLT.10.10.10,Fleet Vehicle Classification,Owned and Leased Distribution|Owned and Leased...,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 11400000 > 11407000 > 11400202 > ...,01/01/24-10053-0030-0010-0010-0010,10053
4,1/1/2024,UBERLANDIA CD - 10096,FLT.10.10.10,Fleet Vehicle Classification,Not Applicable,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 11400000 > 11407000 > 11400202 > ...,01/01/24-10096-0030-0010-0010-0010,10096


In [9]:
text_input_ind["Facility_ID"].unique()

array([10011, 10013, 10031, ..., 93207, 92798, 93064])

In [10]:
# Perform a merge based on Facility_ID
text_input_ind = text_input_ind.merge(
    master_facility[['FACILITY_ID', 'FACILITY_NAME', 'BTLR_NAME_ISSCOM', 'BUNAME_BU']],
    left_on='Facility_ID',
    right_on='FACILITY_ID',
    how='left'
)

# # Drop the extra FACILITY_ID column after merge to avoid duplication
# text_input_ind = text_input_ind.drop(columns=['FACILITY_ID_x'])

# # Rename columns to match the desired names
# text_input_ind = text_input_ind.rename(columns={'FACILITY_ID_y': 'Facility_ID', 'FACILITY_NAME': 'Facility_Name'})


In [11]:
text_input_ind.head()

,Reporting_Period,Entity_Name,Code,Indicator_Name,Answer,Unit,Help_Text,Comments,Frequency,FolderPath,Ord,Facility_ID,FACILITY_ID,FACILITY_NAME,BTLR_NAME_ISSCOM,BUNAME_BU
0,1/1/2024,REYES COCA-COLA BOTTLING RTM NON PRODUCTION - ...,FLT.10.10.10,Fleet Vehicle Classification,Owned and Leased Distribution,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 11300000 > 11300500 > 11300205 > ...,01/01/24-10011-0030-0010-0010-0010,10011,10011.0,REYES COCA-COLA BOTTLING RTM NON PRODUCTION,REYES COCA-COLA BOTTLING,NA BU
1,1/1/2024,HCCB SANAND - 10013,FLT.10.10.10,Fleet Vehicle Classification,Third Party Distribution Vehicles|Owned and Le...,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,01/01/24-10013-0030-0010-0010-0010,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU
2,1/1/2024,TCCC RTM NON PRODUCTION - 10031,FLT.10.10.10,Fleet Vehicle Classification,Not Applicable,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 11300000 > 11300500 > 11300205 > ...,01/01/24-10031-0030-0010-0010-0010,10031,10031.0,TCCC RTM NON PRODUCTION,NaN,NA BU
3,1/1/2024,DUQUE DE CAXIAS - 10053,FLT.10.10.10,Fleet Vehicle Classification,Owned and Leased Distribution|Owned and Leased...,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 11400000 > 11407000 > 11400202 > ...,01/01/24-10053-0030-0010-0010-0010,10053,10053.0,DUQUE DE CAXIAS,ANDINA,LATAM BU
4,1/1/2024,UBERLANDIA CD - 10096,FLT.10.10.10,Fleet Vehicle Classification,Not Applicable,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 11400000 > 11407000 > 11400202 > ...,01/01/24-10096-0030-0010-0010-0010,10096,10096.0,UBERLANDIA CD,UBERLANDIA,LATAM BU


In [12]:
text_input_ind["Answer"].value_counts()

No                                                                                                                                            8878
Not Applicable                                                                                                                                7611
Owned and Leased Distribution|Owned and Leased All Other Vehicles (Company Vans and Cars)                                                     5612
Third Party Distribution Vehicles                                                                                                             3872
Municipality                                                                                                                                  3379
Well                                                                                                                                          2884
Owned and Leased Distribution                                                                                         

In [13]:
# Count the number of missing values in the "Answer" column
missing_count_answer = text_input_ind["Answer"].isnull().sum()

# Print the result
print(f"Number of missing values in 'Answer': {missing_count_answer}")

Number of missing values in 'Answer': 26106


In [14]:
text_input_ind.shape

(70917, 16)

In [15]:
text_input_ind.head()

,Reporting_Period,Entity_Name,Code,Indicator_Name,Answer,Unit,Help_Text,Comments,Frequency,FolderPath,Ord,Facility_ID,FACILITY_ID,FACILITY_NAME,BTLR_NAME_ISSCOM,BUNAME_BU
0,1/1/2024,REYES COCA-COLA BOTTLING RTM NON PRODUCTION - ...,FLT.10.10.10,Fleet Vehicle Classification,Owned and Leased Distribution,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 11300000 > 11300500 > 11300205 > ...,01/01/24-10011-0030-0010-0010-0010,10011,10011.0,REYES COCA-COLA BOTTLING RTM NON PRODUCTION,REYES COCA-COLA BOTTLING,NA BU
1,1/1/2024,HCCB SANAND - 10013,FLT.10.10.10,Fleet Vehicle Classification,Third Party Distribution Vehicles|Owned and Le...,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,01/01/24-10013-0030-0010-0010-0010,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU
2,1/1/2024,TCCC RTM NON PRODUCTION - 10031,FLT.10.10.10,Fleet Vehicle Classification,Not Applicable,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 11300000 > 11300500 > 11300205 > ...,01/01/24-10031-0030-0010-0010-0010,10031,10031.0,TCCC RTM NON PRODUCTION,NaN,NA BU
3,1/1/2024,DUQUE DE CAXIAS - 10053,FLT.10.10.10,Fleet Vehicle Classification,Owned and Leased Distribution|Owned and Leased...,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 11400000 > 11407000 > 11400202 > ...,01/01/24-10053-0030-0010-0010-0010,10053,10053.0,DUQUE DE CAXIAS,ANDINA,LATAM BU
4,1/1/2024,UBERLANDIA CD - 10096,FLT.10.10.10,Fleet Vehicle Classification,Not Applicable,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 11400000 > 11407000 > 11400202 > ...,01/01/24-10096-0030-0010-0010-0010,10096,10096.0,UBERLANDIA CD,UBERLANDIA,LATAM BU


In [16]:
text_input_ind[(text_input_ind['Facility_ID'] == 10013) & (text_input_ind["Indicator_Name"] == "Fleet Vehicle Classification")]

,Reporting_Period,Entity_Name,Code,Indicator_Name,Answer,Unit,Help_Text,Comments,Frequency,FolderPath,Ord,Facility_ID,FACILITY_ID,FACILITY_NAME,BTLR_NAME_ISSCOM,BUNAME_BU
1,1/1/2024,HCCB SANAND - 10013,FLT.10.10.10,Fleet Vehicle Classification,Third Party Distribution Vehicles|Owned and Le...,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,01/01/24-10013-0030-0010-0010-0010,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU
2252,2/1/2024,HCCB SANAND - 10013,FLT.10.10.10,Fleet Vehicle Classification,Third Party Distribution Vehicles|Owned and Le...,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,02/01/24-10013-0030-0010-0010-0010,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU
4506,3/1/2024,HCCB SANAND - 10013,FLT.10.10.10,Fleet Vehicle Classification,Third Party Distribution Vehicles|Owned and Le...,NaN,"Enter ONE OR MORE of the following values, sep...","<p><span style=""font-size: 12.0pt; font-family...",12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,03/01/24-10013-0030-0010-0010-0010,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU
6746,4/1/2024,HCCB SANAND - 10013,FLT.10.10.10,Fleet Vehicle Classification,Third Party Distribution Vehicles|Owned and Le...,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,04/01/24-10013-0030-0010-0010-0010,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU
8994,5/1/2024,HCCB SANAND - 10013,FLT.10.10.10,Fleet Vehicle Classification,Third Party Distribution Vehicles|Owned and Le...,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,05/01/24-10013-0030-0010-0010-0010,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU
11236,6/1/2024,HCCB SANAND - 10013,FLT.10.10.10,Fleet Vehicle Classification,Third Party Distribution Vehicles|Owned and Le...,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,06/01/24-10013-0030-0010-0010-0010,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU
13528,7/1/2024,HCCB SANAND - 10013,FLT.10.10.10,Fleet Vehicle Classification,Third Party Distribution Vehicles|Owned and Le...,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,07/01/24-10013-0030-0010-0010-0010,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU
15824,8/1/2024,HCCB SANAND - 10013,FLT.10.10.10,Fleet Vehicle Classification,Third Party Distribution Vehicles|Owned and Le...,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,08/01/24-10013-0030-0010-0010-0010,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU
18082,9/1/2024,HCCB SANAND - 10013,FLT.10.10.10,Fleet Vehicle Classification,Third Party Distribution Vehicles|Owned and Le...,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,09/01/24-10013-0030-0010-0010-0010,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU
20338,10/1/2024,HCCB SANAND - 10013,FLT.10.10.10,Fleet Vehicle Classification,Third Party Distribution Vehicles|Owned and Le...,NaN,"Enter ONE OR MORE of the following values, sep...",NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,10/01/24-10013-0030-0010-0010-0010,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU


In [17]:
#### Successfully Flags Where Indicator_Name Changes from One Month to the Next
def flag_changes(df):
    # Ensure 'Reporting_Period' is a datetime object for sorting
    df['Reporting_Period'] = pd.to_datetime(df['Reporting_Period'], format='%m/%d/%Y')

    # Sort the DataFrame by 'Facility_ID', 'Indicator_Name', and 'Reporting_Period'
    df.sort_values(by=['Facility_ID', 'Indicator_Name', 'Reporting_Period'], inplace=True)

    # Initialize 'Flag' column with default value 0
    df['Flag'] = 0

    # Identify where the 'Answer' field changes for each 'Facility_ID' and 'Indicator_Name' combination
    df['Flag'] = df.groupby(['Facility_ID', 'Indicator_Name'])['Answer'].transform(
        lambda x: x != x.shift())

    # Convert boolean to integer (0, 1) for the 'Flag' column
    df['Flag'] = df['Flag'].astype(int)

    # Return the modified DataFrame
    return df

# Example usage
# Assuming 'text_input_ind' is your DataFrame
text_flag = flag_changes(text_input_ind)

text_flag.head()

,Reporting_Period,Entity_Name,Code,Indicator_Name,Answer,Unit,Help_Text,Comments,Frequency,FolderPath,Ord,Facility_ID,FACILITY_ID,FACILITY_NAME,BTLR_NAME_ISSCOM,BUNAME_BU,Flag
27277,2024-01-01,BUKAVU - 12,EC.10.10.180,Do you have secondary operations on-site? (ie....,No,NaN,Enter ONE of the following values: Yes|No,NaN,12,ALL > TCCS > 21000100 > 21000210 > 21000710 > ...,01/01/24-12-0010-0040-0010-0180,12,12.0,BUKAVU,HEINEKEN,AFRICA BU,1
30217,2024-02-01,BUKAVU - 12,EC.10.10.180,Do you have secondary operations on-site? (ie....,No,NaN,Enter ONE of the following values: Yes|No,NaN,12,ALL > TCCS > 21000100 > 21000210 > 21000710 > ...,02/01/24-12-0010-0040-0010-0180,12,12.0,BUKAVU,HEINEKEN,AFRICA BU,0
33145,2024-03-01,BUKAVU - 12,EC.10.10.180,Do you have secondary operations on-site? (ie....,No,NaN,Enter ONE of the following values: Yes|No,NaN,12,ALL > TCCS > 21000100 > 21000210 > 21000710 > ...,03/01/24-12-0010-0040-0010-0180,12,12.0,BUKAVU,HEINEKEN,AFRICA BU,0
36064,2024-04-01,BUKAVU - 12,EC.10.10.180,Do you have secondary operations on-site? (ie....,No,NaN,Enter ONE of the following values: Yes|No,NaN,12,ALL > TCCS > 21000100 > 21000210 > 21000710 > ...,04/01/24-12-0010-0040-0010-0180,12,12.0,BUKAVU,HEINEKEN,AFRICA BU,0
38992,2024-05-01,BUKAVU - 12,EC.10.10.180,Do you have secondary operations on-site? (ie....,No,NaN,Enter ONE of the following values: Yes|No,NaN,12,ALL > TCCS > 21000100 > 21000210 > 21000710 > ...,05/01/24-12-0010-0040-0010-0180,12,12.0,BUKAVU,HEINEKEN,AFRICA BU,0


In [18]:
def update_detection_result(df):
    # Ensure 'Reporting_Period' is a datetime object for sorting if not already done
    df['Reporting_Period'] = pd.to_datetime(df['Reporting_Period'], format='%m/%d/%Y')

    # Sort the DataFrame appropriately
    df.sort_values(by=['Facility_ID', 'Indicator_Name', 'Reporting_Period'], inplace=True)

    # Create the Detection_Result column with default NaN
    df['Detection_Result'] = pd.NA

    # Iterate over each group of 'Facility_ID' and 'Indicator_Name'
    for name, group in df.groupby(['Facility_ID', 'Indicator_Name']):
        # Check where the Answer changed and Flag is 1
        changes = group.index[group['Flag'] == 1]

        for idx in changes:
            # Establishing previous and current indices for text generation
            prev_idx = idx - 1
            if prev_idx in df.index:
                facility_id = df.at[idx, 'Facility_ID']
                new_indicator_name = df.at[idx, 'Indicator_Name']
                prev_indicator_name = df.at[prev_idx, 'Indicator_Name']
                detection_text = f"Facility #: {facility_id}, Indicator: {new_indicator_name} switched from {prev_indicator_name} to {new_indicator_name}"
                df.at[idx, 'Detection_Result'] = detection_text

    # Return updated DataFrame
    return df

In [19]:
# Example usage
tmp = update_detection_result(text_flag)
tmp[(tmp['Flag'] == 1) & (tmp['Facility_ID'] == 10013)]

,Reporting_Period,Entity_Name,Code,Indicator_Name,Answer,Unit,Help_Text,Comments,Frequency,FolderPath,Ord,Facility_ID,FACILITY_ID,FACILITY_NAME,BTLR_NAME_ISSCOM,BUNAME_BU,Flag,Detection_Result
27223,2024-01-01,HCCB SANAND - 10013,EC.10.10.180,Do you have secondary operations on-site? (ie....,No,NaN,Enter ONE of the following values: Yes|No,NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,01/01/24-10013-0010-0040-0010-0180,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU,1,"Facility #: 10013, Indicator: Do you have seco..."
27225,2024-01-01,HCCB SANAND - 10013,EC.20.10.35,Electricity is paid by TCCS?,NaN,NaN,Fill in an Answer only if the answer to 'Selec...,NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,01/01/24-10013-0010-0050-0010-0035,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU,1,"Facility #: 10013, Indicator: Electricity is p..."
30165,2024-02-01,HCCB SANAND - 10013,EC.20.10.35,Electricity is paid by TCCS?,NaN,NaN,Fill in an Answer only if the answer to 'Selec...,NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,02/01/24-10013-0010-0050-0010-0035,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU,1,"Facility #: 10013, Indicator: Electricity is p..."
33093,2024-03-01,HCCB SANAND - 10013,EC.20.10.35,Electricity is paid by TCCS?,NaN,NaN,Fill in an Answer only if the answer to 'Selec...,NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,03/01/24-10013-0010-0050-0010-0035,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU,1,"Facility #: 10013, Indicator: Electricity is p..."
36012,2024-04-01,HCCB SANAND - 10013,EC.20.10.35,Electricity is paid by TCCS?,NaN,NaN,Fill in an Answer only if the answer to 'Selec...,NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,04/01/24-10013-0010-0050-0010-0035,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU,1,"Facility #: 10013, Indicator: Electricity is p..."
38940,2024-05-01,HCCB SANAND - 10013,EC.20.10.35,Electricity is paid by TCCS?,NaN,NaN,Fill in an Answer only if the answer to 'Selec...,NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,05/01/24-10013-0010-0050-0010-0035,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU,1,"Facility #: 10013, Indicator: Electricity is p..."
41853,2024-06-01,HCCB SANAND - 10013,EC.20.10.35,Electricity is paid by TCCS?,NaN,NaN,Fill in an Answer only if the answer to 'Selec...,NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,06/01/24-10013-0010-0050-0010-0035,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU,1,"Facility #: 10013, Indicator: Electricity is p..."
44787,2024-07-01,HCCB SANAND - 10013,EC.20.10.35,Electricity is paid by TCCS?,NaN,NaN,Fill in an Answer only if the answer to 'Selec...,NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,07/01/24-10013-0010-0050-0010-0035,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU,1,"Facility #: 10013, Indicator: Electricity is p..."
47721,2024-08-01,HCCB SANAND - 10013,EC.20.10.35,Electricity is paid by TCCS?,NaN,NaN,Fill in an Answer only if the answer to 'Selec...,NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,08/01/24-10013-0010-0050-0010-0035,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU,1,"Facility #: 10013, Indicator: Electricity is p..."
50634,2024-09-01,HCCB SANAND - 10013,EC.20.10.35,Electricity is paid by TCCS?,NaN,NaN,Fill in an Answer only if the answer to 'Selec...,NaN,12,ALL > TCCS > 19000300 > 11700600 > 11700602 > ...,09/01/24-10013-0010-0050-0010-0035,10013,10013.0,HCCB SANAND,HINDUSTAN CC BVG,INSWA BU,1,"Facility #: 10013, Indicator: Electricity is p..."
